In [4]:
import pandas as pd


In [5]:
data_fuel_events = pd.read_csv('data/Extranet2.ExactFuelEvents.csv')

In [49]:
data_fuel_tank_levels = pd.read_csv('data/Extranet2.ExactFuelTankLevels.csv')

In [7]:
data_fuel_transaction = pd.read_csv('data/BGETCHDATA.PTCHTRANH.csv')

In [8]:
data_fuel_transaction.shape

(147861, 21)

In [9]:
data_units = pd.read_csv('data/IBGEFILE.UNITS.csv')

In [10]:
data_units.shape

(709, 3)

In [11]:
data_units.columns = ['unit_number', 'model_year', 'make']

In [12]:
data_units.head(n=5)

,unit_number,model_year,make
0,535,1996,PETERBILT
1,534,1996,PETERBILT
2,533,1996,PETERBILT
3,532,1996,PETERBILT
4,531,1996,PETERBILT


In [13]:
data_fuel_optimization = pd.read_csv('data/BGEIDSC.EF2EFFP.csv')

In [14]:
data_fuel_optimization.shape

(836, 7)

In [15]:
#column names to lower for easier typing
data_fuel_optimization.columns = [s.lower() for s in data_fuel_optimization.columns]
data_fuel_optimization.columns

Index(['truck', 'active', 'sendfuel', 'sendroute', 'tankcap', 'avgmpg',
       'sattype'],
      dtype='object')

In [54]:
data_fuel_optimization[data_fuel_optimization.tankcap != 200]

,truck,active,sendfuel,sendroute,tankcap,avgmpg,sattype
5,1330,Y,Y,Y,230,6.0,
6,1334,Y,Y,Y,230,6.0,
7,1336,Y,Y,Y,230,6.0,
8,1340,Y,Y,Y,230,6.0,
9,1341,Y,Y,Y,230,6.0,
10,1345,Y,Y,Y,230,6.0,
11,1348,Y,Y,Y,230,6.0,
12,1352,Y,Y,Y,230,6.0,
13,1354,Y,Y,Y,230,6.0,
14,1357,Y,Y,Y,230,6.0,


In [17]:
#Actual fuel transactions will have an item code of:
act_fuel_transactions = ['ULSD', 'FUEL', 'CDSL', 'DSL1', 'BDSL']
# and could be in any of the item slots 1-6

In [18]:
data_fuel_transaction.columns = ['tr_id', 'tr_date', 'tr_time', 'tr_station_id', 'tr_station_name',
                         'tr_station_cty', 'tr_station_st', 'tr_drvr_code', 'tr_unit_id', 
                         'tr_item_cd1', 'tr_item_qty1', 'tr_item_cd2', 'tr_item_qty2',
                          'tr_item_cd3', 'tr_item_qty3', 'tr_item_cd4', 'tr_item_qty4',
                          'tr_item_cd5', 'tr_item_qty5', 'tr_item_cd6', 'tr_item_qty6',
                         ]

In [19]:
data_fuel_transaction.head(n=5)

,tr_id,tr_date,tr_time,tr_station_id,tr_station_name,tr_station_cty,tr_station_st,tr_drvr_code,tr_unit_id,tr_item_cd1,...,tr_item_cd2,tr_item_qty2,tr_item_cd3,tr_item_qty3,tr_item_cd4,tr_item_qty4,tr_item_cd5,tr_item_qty5,tr_item_cd6,tr_item_qty6
0,42574765,20170101,20,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,...,,0.00,,0.0,,0.0,,0.0,,0
1,42575331,20170101,48,516202,PILOT NEW BRAUNFELS 330,NEW BRAUNFELS,TX,MANRI,1584,ULSD,...,DEFD,7.71,CADV,1.0,,0.0,,0.0,,0
2,42578678,20170101,134,516277,PILOT 421,DALTON,GA,RUSR,1713,ULSD,...,DEFD,4.12,,0.0,,0.0,,0.0,,0
3,42578722,20170101,137,231100,FJ-BRUNSWICK 627,BRUNSWICK,GA,FOSJA,1711,ULSD,...,,0.00,,0.0,,0.0,,0.0,,0
4,42579695,20170101,302,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,HICWIL,1837,ULSD,...,,0.00,,0.0,,0.0,,0.0,,0


In [20]:
data_fuel_transaction.shape

(147861, 21)

In [21]:
data_fuel_events.head(n=5)

,Id,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Latitude,Longitude,LocationTimeStamp,Speed,Heading,Odometer,IgnitionStatus,EFReportReason
0,12649083,13640241,2017-01-01 00:00:32.387,1992,105369677,tractor,BATK,30.34642,-81.70724,2017-01-01 00:00:30.000,0,0.0,39895.5,2,1
1,12649084,13640242,2017-01-01 00:01:24.867,1712,105437938,tractor,TAYANT,36.93953,-84.09541,2017-01-01 00:01:27.000,0,0.0,360378.8,1,2
2,12649092,13640251,2017-01-01 00:04:15.053,1713,105438293,tractor,RUSR,35.58664,-84.52786,2017-01-01 00:04:17.000,67,219.5,377577.4,1,2
3,12649102,13640261,2017-01-01 00:04:10.260,1585,105301976,tractor,MCAD,35.15430,-90.14263,2017-01-01 00:04:10.000,0,0.0,513931.8,1,2
4,12649103,13640272,2017-01-01 00:10:13.187,1564,105351785,tractor,ULAL,39.90268,-84.19254,2017-01-01 00:10:11.000,3,91.0,498691.0,1,2


In [22]:
data_fuel_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8495130 entries, 0 to 8495129
Data columns (total 15 columns):
Id                   int64
ESS_Id               int64
EventTimeStamp       object
EquipmentID          int64
MCTNumber            int64
EquipmentType        object
DriverID             object
Latitude             float64
Longitude            float64
LocationTimeStamp    object
Speed                int64
Heading              float64
Odometer             float64
IgnitionStatus       int64
EFReportReason       int64
dtypes: float64(4), int64(7), object(4)
memory usage: 972.2+ MB


In [23]:
data_fuel_events.isnull().any()

Id                   False
ESS_Id               False
EventTimeStamp       False
EquipmentID          False
MCTNumber            False
EquipmentType        False
DriverID              True
Latitude             False
Longitude            False
LocationTimeStamp    False
Speed                False
Heading              False
Odometer             False
IgnitionStatus       False
EFReportReason       False
dtype: bool

In [24]:
type(data_fuel_events)

pandas.core.frame.DataFrame

In [25]:
data_fuel_events['DriverID'].value_counts()

DYET          33901
SIMJIM        27204
VAUJAM        25743
ULAL          25584
ECKS          24125
CHAJO         23895
OBRG          23551
BULR          23042
CUNW          22963
RENT          22957
LOVH          22395
BROD03        22313
PERT          22190
FARMI         22039
STEGA         21968
CHRN          21921
NARJ          21574
GALJU         21530
SWEJ          21140
ANTR          21046
TAYJON        20882
EDWTI         20845
SAVE          20688
FERCA         20550
SIMJE         20529
STOD          20443
DINE          20246
MORJAM        20216
CRES          19957
JENLE         19871
              ...  
SHOP3            52
BANJA            51
DENJE            48
CARJ01           48
REERO            42
STAW             41
SNID             22
ADADE            20
RUDD             17
MEID             12
JOHLO            11
SCHJ             10
CRAIG B          10
HARGR             2
SHOP!             2
BYRCHI            2
WILRON            2
GODJO             2
KINCA             2


In [26]:
data_fuel_events['Odometer'].max()

861003.90000000002

In [28]:
data_fuel_events.loc[data_fuel_events['Odometer'] == 861003.90000000002]


,Id,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Latitude,Longitude,LocationTimeStamp,Speed,Heading,Odometer,IgnitionStatus,EFReportReason
7829068,20484418,40248138,2018-02-17 11:31:56.697,1664,105438725,tractor,HERD,33.02991,-84.96272,2018-02-17 11:31:57.000,0,289.7,861003.9,1,1


In [50]:
len(data_fuel_events['DriverID'].unique())

949

In [30]:
data_fuel_events['Speed'].value_counts()

0      5297851
67      329702
2       169310
65      159735
66      146349
3       134205
4       128776
1       110966
5       104669
6        92148
64       85300
7        80793
8        75987
63       66388
9        62457
62       60094
68       59968
61       55696
60       55437
10       54146
59       50708
58       48090
11       47974
57       44326
56       41349
12       38793
55       37598
54       32557
13       30631
52       28734
        ...   
38       15817
27       15609
26       15309
32       15074
34       14991
24       14911
23       14900
33       14864
36       14822
37       14809
25       14773
35       14596
70       12334
71        7497
72        4488
73        2680
74        1494
75         935
76         486
158        390
77         263
78         151
79          61
80          42
81          21
83          12
82          11
84           3
88           1
143          1
Name: Speed, Length: 88, dtype: int64

In [52]:
data_fuel_tank_levels.head(n=5)

,Id,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
0,10391480,1,82.8,165.60,12667508
1,10391482,1,72.4,144.80,12667510
2,10391505,1,57.1,114.20,12667533
3,10391507,1,80.9,161.80,12667535
4,10391514,1,56.9,142.25,12667542


In [53]:
data_fuel_tank_levels[data_fuel_tank_levels.TankId == 2]

,Id,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
48670,10633484,2,5.5,11.00,12933194
1824799,14078949,2,1.8,3.60,16673494
4579414,13743578,2,5.5,11.00,16313459
5656380,13899589,2,1.8,4.14,16481275


In [20]:
data_fuel_tank_levels.isnull().any()

Id                  False
TankId              False
TankLevelPercent    False
TankLevelGallons    False
ExactFuelEventId    False
dtype: bool

In [21]:
data_fuel_events.isnull().sum()

Id                        0
ESS_Id                    0
EventTimeStamp            0
EquipmentID               0
MCTNumber                 0
EquipmentType             0
DriverID             318942
Latitude                  0
Longitude                 0
LocationTimeStamp         0
Speed                     0
Heading                   0
Odometer                  0
IgnitionStatus            0
EFReportReason            0
dtype: int64

In [19]:
print(data_units.head(n=15))

   UNUNIT UNYEAR        UNMAKE
0     535   1996  PETERBILT   
1     534   1996  PETERBILT   
2     533   1996  PETERBILT   
3     532   1996  PETERBILT   
4     531   1996  PETERBILT   
5     526   1996  PETERBILT   
6     238   1995  PETERBILT   
7     536   1997  PETERBILT   
8     537   1997  PETERBILT   
9     538   1997  PETERBILT   
10    539   1997  PETERBILT   
11    540   1997  PETERBILT   
12      1   1996              
13      1   1900  SYSTEM TEST 
14    121   1999  PETERBILT   


In [39]:
sorted(data_units['UNMAKE'].unique())

['            ',
 'FREIGHTLINER',
 'KENOWRTH    ',
 'KENWORTH    ',
 'KEWORTH     ',
 'MACK        ',
 'PETERBILT   ',
 'SYSTEM TEST ',
 'VOLVO       ']

In [38]:
data_units.loc[data_units['UNYEAR'] == '2008']

,UNUNIT,UNYEAR,UNMAKE
22,245,2008,VOLVO
23,246,2008,VOLVO


In [43]:
data_fuel_optimization.head(n=5)

,TRUCK,ACTIVE,SENDFUEL,SENDROUTE,TANKCAP,AVGMPG,SATTYPE
0,1,Y,Y,Y,200,6.0,
1,243,Y,Y,Y,200,6.0,
2,244,Y,Y,Y,200,6.0,
3,245,Y,Y,Y,200,6.0,
4,246,Y,Y,Y,200,6.0,


In [44]:
sorted(data_fuel_optimization['AVGMPG'].unique())

[6.0]

In [47]:
data_fuel_optimization.groupby(['TANKCAP']).count()

,TRUCK,ACTIVE,SENDFUEL,SENDROUTE,AVGMPG,SATTYPE
TANKCAP,,,,,,
120,4,4,4,4,4,4
200,510,510,510,510,510,510
230,180,180,180,180,180,180
235,10,10,10,10,10,10
250,132,132,132,132,132,132


In [49]:
data_fuel_transaction.head(n=5)

,TRNID,TRNDAT,TRNTIM,TRNTS#,TRNTSN,TRNSTC,TRNST,TRNDRI,TRNUNT,TRNPI1,...,TRNPI2,TRNPQ2,TRNPI3,TRNPQ3,TRNPI4,TRNPQ4,TRNPI5,TRNPQ5,TRNPI6,TRNPQ6
0,42574765,20170101,20,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,...,,0.00,,0.0,,0.0,,0.0,,0
1,42575331,20170101,48,516202,PILOT NEW BRAUNFELS 330,NEW BRAUNFELS,TX,MANRI,1584,ULSD,...,DEFD,7.71,CADV,1.0,,0.0,,0.0,,0
2,42578678,20170101,134,516277,PILOT 421,DALTON,GA,RUSR,1713,ULSD,...,DEFD,4.12,,0.0,,0.0,,0.0,,0
3,42578722,20170101,137,231100,FJ-BRUNSWICK 627,BRUNSWICK,GA,FOSJA,1711,ULSD,...,,0.00,,0.0,,0.0,,0.0,,0
4,42579695,20170101,302,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,HICWIL,1837,ULSD,...,,0.00,,0.0,,0.0,,0.0,,0


In [60]:
data_fuel_transaction.iloc[5:20,5:]

,TRNSTC,TRNST,TRNDRI,TRNUNT,TRNPI1,TRNPQ1,TRNPI2,TRNPQ2,TRNPI3,TRNPQ3,TRNPI4,TRNPQ4,TRNPI5,TRNPQ5,TRNPI6,TRNPQ6
5,ST MARYS,GA,BATK,1992,ULSD,93.88,,0.00,,0.0,,0.0,,0.0,,0
6,ST MARYS,GA,BATK,1992,SCLE,1.00,,0.00,,0.0,,0.0,,0.0,,0
7,GAFFNEY,SC,MARJOH,1730,ULSD,136.29,DEFD,5.50,,0.0,,0.0,,0.0,,0
8,CHARLOTTE,NC,JAML,1672,ULSD,149.04,,0.00,,0.0,,0.0,,0.0,,0
9,CHARLOTTE,NC,JAML,1672,CADV,1.00,,0.00,,0.0,,0.0,,0.0,,0
10,KNOXVILLE,TN,BEAKEV,1739,ULSD,99.85,CADV,1.00,,0.0,,0.0,,0.0,,0
11,JACKSON,TN,MCAD,1585,ULSD,131.69,CADV,1.00,,0.0,,0.0,,0.0,,0
12,GLADEVILLE,TN,JARJ,1930,ULSD,90.62,,0.00,,0.0,,0.0,,0.0,,0
13,MOUNT AIRY,NC,LESJ,1608,ULSD,89.72,DEFD,3.58,CADV,1.0,,0.0,,0.0,,0
14,MORGANTOWN,WV,BLACH,1878,ULSD,61.52,,0.00,,0.0,,0.0,,0.0,,0


In [52]:
data_fuel_transaction['TRNST'].unique()

array(['OH', 'TX', 'GA', 'TN', 'SC', 'NC', 'WV', 'IN', 'AR', 'VA', 'PA',
       'KY', 'AL', 'FL', 'NJ', 'LA', 'IL', 'MS', 'MD', 'MA', 'MI', 'WI',
       'MN', 'MO', 'NY', 'OK', 'CT', 'IA', 'NM', 'AZ', 'NV', 'RI', 'KS',
       'NH', 'CO', 'WY', 'UT', 'NE', 'CA', 'ME', 'ID', 'MT', 'SD', 'WA',
       'OR'], dtype=object)

In [102]:
data_fuel_transaction.groupby(['TRNST','TRNSTC','TRNTSN']).size()

TRNST  TRNSTC           TRNTSN                   
AL     BIRMINGHAM       CAT SCALES FLYING J / PIL      8
                        COWBOYS #3672                  1
                        FJ-BIRMINGHAM 602            340
                        PILOT BIRMINGHAM 369          60
       BOAZ             COWBOYS #26                    1
       BRENT            SUPER MART                     1
       BREWTON          KANGAROO EXPRESS #1571         1
       BUCKSVILLE       CAT SCALES PETRO STOPPING      1
       CALERA           SPEED TRACK INC                1
       CHILDERSBURG     COWBOYS #3664                  1
       CLANTON          CAT SCALES LOVE'S COUNTRY      9
                        LOVES # 368 TRAVEL STOP      153
                        THE STORE #1                   1
       COTTONDALE       CAT SCALES  TA COTTONDALE      1
                        PFJ SOUTHEAST 4555             2
                        PILOT #4555                   48
                        PILOT FLYING J

In [63]:
data_fuel_events.head(n=5)

,Id,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Latitude,Longitude,LocationTimeStamp,Speed,Heading,Odometer,IgnitionStatus,EFReportReason
0,12649083,13640241,2017-01-01 00:00:32.387,1992,105369677,tractor,BATK,30.34642,-81.70724,2017-01-01 00:00:30.000,0,0.0,39895.5,2,1
1,12649084,13640242,2017-01-01 00:01:24.867,1712,105437938,tractor,TAYANT,36.93953,-84.09541,2017-01-01 00:01:27.000,0,0.0,360378.8,1,2
2,12649092,13640251,2017-01-01 00:04:15.053,1713,105438293,tractor,RUSR,35.58664,-84.52786,2017-01-01 00:04:17.000,67,219.5,377577.4,1,2
3,12649102,13640261,2017-01-01 00:04:10.260,1585,105301976,tractor,MCAD,35.15430,-90.14263,2017-01-01 00:04:10.000,0,0.0,513931.8,1,2
4,12649103,13640272,2017-01-01 00:10:13.187,1564,105351785,tractor,ULAL,39.90268,-84.19254,2017-01-01 00:10:11.000,3,91.0,498691.0,1,2


In [70]:
data_fuel_events['Speed'].unique()

array([  0,  67,   3,  53,  56,  10,  46,  63,   9,  57,  13,   2,  16,
         6,  48,  65,  66,  31,   8,  47,  64,   7,  69,  61,  59,  42,
         1,   4,  60,  14,  51,  17,  72,  12,  55,  58,  21,  26,  62,
        24,  33,   5,  38,  15,  41,  44,  11,  25,  30,  23,  27,  39,
        68,  54,  52,  18,  70,  19,  43,  71,  32,  34,  28,  45,  50,
        75,  35,  22,  36,  49,  37,  20,  40,  29,  73,  76,  74,  77,
        79,  81,  83,  84, 158,  82,  78,  80, 143,  88], dtype=int64)

In [84]:
data_fuel_events.groupby(['DriverID','Speed']).size().to_frame('size').reset_index().sort_values(['DriverID', 'size'], ascending=[True, False])
#data_fuel_transaction.groupby(['TRNST','TRNSTC','TRNTSN']).count()

,DriverID,Speed,size
0,36728,0,1
1,ABBV,0,3365
68,ABBV,67,378
3,ABBV,2,173
67,ABBV,66,132
5,ABBV,4,116
4,ABBV,3,105
2,ABBV,1,98
9,ABBV,8,95
66,ABBV,65,94


In [73]:
data_fuel_events[data_fuel_events['DriverID'] == 'BATK']

,Id,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Latitude,Longitude,LocationTimeStamp,Speed,Heading,Odometer,IgnitionStatus,EFReportReason
0,12649083,13640241,2017-01-01 00:00:32.387,1992,105369677,tractor,BATK,30.34642,-81.70724,2017-01-01 00:00:30.000,0,0.0,39895.5,2,1
8,12649107,13640276,2017-01-01 00:13:02.450,1992,105369677,tractor,BATK,30.34646,-81.70727,2017-01-01 00:13:00.000,0,0.0,39895.5,1,0
11,12649110,13640280,2017-01-01 00:15:52.450,1992,105369677,tractor,BATK,30.34605,-81.70736,2017-01-01 00:15:50.000,0,0.0,39895.6,2,1
17,12649116,13640292,2017-01-01 00:26:22.497,1992,105369677,tractor,BATK,30.34603,-81.70738,2017-01-01 00:26:20.000,0,0.0,39895.6,1,0
33,12649132,13640316,2017-01-01 00:41:22.560,1992,105369677,tractor,BATK,30.36038,-81.76080,2017-01-01 00:41:20.000,63,359.3,39901.1,1,2
89,12649191,13640352,2017-01-01 01:02:42.683,1992,105369677,tractor,BATK,30.61811,-81.64883,2017-01-01 01:02:41.000,0,0.0,39923.4,1,2
103,12649205,13640378,2017-01-01 01:17:42.810,1992,105369677,tractor,BATK,30.62091,-81.52817,2017-01-01 01:17:41.000,48,107.0,39930.8,1,2
118,12649220,13640398,2017-01-01 01:30:02.827,1992,105369677,tractor,BATK,30.67889,-81.45789,2017-01-01 01:30:01.000,0,0.0,39938.3,2,1
124,12649230,13640413,2017-01-01 01:34:02.840,1992,105369677,tractor,BATK,30.67917,-81.45939,2017-01-01 01:34:01.000,0,0.0,39938.4,2,1
125,12649231,13640414,2017-01-01 01:34:32.840,1992,105369677,tractor,BATK,30.67917,-81.45943,2017-01-01 01:34:31.000,0,0.0,39938.4,2,0


In [74]:
data_fuel_tank_levels.head(n=5)

,Id,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
0,10391480,1,82.8,165.60,12667508
1,10391482,1,72.4,144.80,12667510
2,10391505,1,57.1,114.20,12667533
3,10391507,1,80.9,161.80,12667535
4,10391514,1,56.9,142.25,12667542


In [76]:
data_fuel_tank_levels['TankId'].value_counts()

1    7908824
2          4
Name: TankId, dtype: int64

In [79]:
data_fuel_tank_levels[data_fuel_tank_levels['TankId'] == 2]

,Id,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
48670,10633484,2,5.5,11.00,12933194
1824799,14078949,2,1.8,3.60,16673494
4579414,13743578,2,5.5,11.00,16313459
5656380,13899589,2,1.8,4.14,16481275


In [86]:
# Datetime format reference: 
# https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior 
##########---------converting cols to datetime-------

def cols_to_timestamp(df, col_list):
    for col in col_list:
        df[col] = pd.to_datetime(df[col], format = '%Y-%m-%d %H:%M:%S')
    return df
#ts_cols = ['EventTimeStamp', 'LocationTimeStamp']
#events_fuel_df =  cols_to_timestamp(events_fuel_df, ts_cols) 

In [93]:
#data performance 
data_performance = pd.read_csv('data/Extranet2.QCPerformanceExtracts.csv',header= None)


In [100]:
data_performance.tail(n=5)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
32382,116174,35853279,2017-12-30 13:17:05.000,1854,105385876,tractor,ROSE,NaN,35.280046,-80.806249,...,0,233.8,5.7,0,1,0,1607,3,MPH,"35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."
32383,116175,35856910,2017-12-30 14:18:35.000,1979,105313312,tractor,HIEK,NaN,36.066296,-86.434537,...,0,0.5,0.3,0,0,0,0,3,MPH,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
32384,116176,35856911,2017-12-30 14:18:36.000,1979,105313312,tractor,HIEK,NaN,36.066296,-86.434537,...,0,235.2,2.0,0,1,527,0,3,MPH,"34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."
32385,116177,35874186,2017-12-30 23:33:46.000,2028,108034857,tractor,JORW,NaN,34.193148,-80.282499,...,0,211.6,18.2,0,1,254,1186,3,MPH,"107,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
32386,116178,35874249,2017-12-30 23:37:47.000,2090,108087387,tractor,SHADE,NaN,35.050648,-85.636620,...,0,311.7,0.6,0,1,8,1816,1,MPH,"70,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."


In [95]:
data_performance.info

<bound method DataFrame.info of            0         1                        2          3          4   \
0       76818  13641356  2017-01-01 07:29:49.000       1995  105356577   
1       76819  13641357  2017-01-01 07:29:50.000       1995  105356577   
2       76820  13641432  2017-01-01 07:49:44.000  105420005  105420005   
3       76821  13643644  2017-01-01 13:58:17.000       1972  105363528   
4       76822  13643645  2017-01-01 13:58:18.000       1972  105363528   
5       76823  13643777  2017-01-01 14:19:55.000       1985  105370056   
6       76824  13644321  2017-01-01 16:00:28.000       1861  105339124   
7       76825  13644322  2017-01-01 16:00:29.000       1861  105339124   
8       76826  13644956  2017-01-01 18:09:40.000       1538  105392945   
9       76827  13645006  2017-01-01 18:18:19.000       1777  105446111   
10      76828  13645007  2017-01-01 18:18:19.000       1777  105446111   
11      76829  13645204  2017-01-01 19:21:44.000       1814  105369518   
12    

In [98]:
data_performance.shape

(32387, 32)

In [99]:
data_performance.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
           dtype='int64')